# <span style="color:#9B59B6"> SPRINT 3: Modelo de classificação de sentimentos: positivos ou negativos</span>

Equipe <b>Da Vinci Insights</b>:

Debora Damaso Lopes

Cassio Yuji Hirassike Sakai

Paulo Barbosa Neto

Yasmin Lopes

Allef Santos



# <span style="color:#9B59B6"> 1. Sumário </span> <a class="ancora" id="sumario_1"></a>

Este notebook tem como objetivo o treinamento de um modelo de classificação binária que identifica sentimentos positivos e negativos, com base em avaliações e comentários de consumidores sobre produtos comprados. Os modelos utilizados foram Logistic Regression e Random Forest. Corresponde à entrega da disciplina Disruptive Architectures: IOT, IOB & Generative IA.

# <span style="color:#9B59B6"> 2. Descrição de classificação </span> <a class="ancora" id="sumario_1"></a>


Classificação refere-se à categorização dos dados fornecidos em classes.

Por exemplo:

* Dada uma imagem de um caractere escrito à mão, identificar o caractere (classificação multi-classe);

* Dada uma imagem, anotá-la com todos os objetos presentes na imagem (classificação multi-rótulo);

* Classificar um e-mail como spam ou não spam (classificação binária);

* Classificar um tumor como benigno ou maligno e assim por diante.


# <span style="color:#9B59B6">  3. Proposta de negócio </span>  

A Da Vinci Insights visa implementar modelos de IA em suas plataformas, tanto web quanto mobile, para auxiliar empresas à identificarem o nível de satisfação dos consumidores sobre seus produtos, assim como pontos de melhorias e impulsionamento do produto avaliado em si, ou da quantidade de avaliações. Mais detalhes podem ser consultados no pdf <i>DocumentacaoGeral.pdf</i> dentro desta <a href="https://drive.google.com/drive/folders/1w7pozCVdJworLfcloYjfBSoIqhi4SL99?usp=sharing">pasta do google drive</a>.


## <span style="color:#9B59B6"> 4. Etapa de Preparação

#### 4.1 Datasets utilizados:
O Dataset <a href="https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce">Brazilian E-Commerce Public Dataset by Olist</a> utilizado foi retirado do site Kaggle. Se trata de um Dataset com o índice de usabilidade de 10 calculado pelo Kaggle.


#### 4.2 Organização dos Dados:

* O arquivo base do estudo se chama <i>olist_order_reviews_dataset.csv</i> e foi extraído do dataset para dentro desta <a href="https://drive.google.com/drive/folders/1w7pozCVdJworLfcloYjfBSoIqhi4SL99?usp=sharing">pasta do google drive</a>, na subpasta <i>archive</i>.

#### 4.3 Objetivo deste notebook:

* Este notebook <b>TreinamentoModelo.ipynb</b> é focado apenas no treinamento deste modelo de sentimentos em si, que é o objetivo da Sprint 3. Posteriormente, outro modelo será treinado na fase seguinte (Sprint 4) para analisar outros aspectos como dados de um dataset em relação aos produtos.

Para entender melhor o tipo de dataset que estamos lidando, poderá visualizar uma análise sobre este dataset no arquivo <b>AnaliseDataset.ipynb</b>, também dentro da pasta do drive.


## <span style="color:#9B59B6"> 5. Bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
import re

In [4]:
# Carregar o dataset de reviews da Olist
reviews = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DaVinci_Sprint3/archive/olist_order_reviews_dataset.csv')

# Visualizar as primeiras linhas do dataset
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


### **Entendimento do Dataset**
O dataset da Olist tem várias tabelas. A principal para a análise de avaliações de clientes é a tabela **`olist_order_reviews_dataset`**, que contém as avaliações que os clientes deram para os pedidos. Essa tabela possui as seguintes colunas relevantes:

- **`review_id`**: Identificador único da avaliação.
- **`order_id`**: Relaciona-se ao pedido.
- **`review_score`**: Nota dada pelo cliente (de 1 a 5).
- **`review_comment_title`**: Título da avaliação (pode estar vazio).
- **`review_comment_message`**: Mensagem da avaliação (texto livre).
- **`review_creation_date`**: Data da criação da avaliação.
- **`review_answer_timestamp`**: Data de resposta da avaliação pela Olist.

## <span style="color:#9B59B6"> 6. Pré-processamento de Dados

#### 6.1 Processamento de texto:

In [5]:
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

def preprocess_text(text):
    # Remover caracteres especiais e colocar tudo em minúsculas
    text = re.sub(r'[^a-zA-Z\s]', '', str(text).lower())
    # Remover stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Aplicar a função de pré-processamento nos comentários
reviews['review_comment_message'] = reviews['review_comment_message'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### 6.2 Converter as notas de avaliação em apenas sentimentos positivos e negativos:

In [6]:
# Definir sentimento com base nas notas de avaliação (1-2: negativo, 4-5: positivo)
def sentiment_label(rating):
    if rating in [1, 2]:
        return 'negativo'
    elif rating in [4, 5]:
        return 'positivo'
    else:
        return None  # Ignorar notas 3

# Aplicar a função às avaliações
reviews['sentiment'] = reviews['review_score'].apply(sentiment_label)

# Remover as linhas onde o sentimento é None (avaliação neutra)
reviews = reviews.dropna(subset=['sentiment'])

#### 6.3 Remoção de valores nulos e Divisão em dados de Treino e Teste

In [7]:
# Remover valores nulos nos comentários
reviews = reviews.dropna(subset=['review_comment_message'])

# Separar as features (comentários) e o rótulo (sentimento)
X = reviews['review_comment_message']
y = reviews['sentiment']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### 6.4 Vetorização dos comentários usando TF-IDF:

In [8]:
# Transformar os textos em vetores TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

## <span style="color:#9B59B6"> 7. Treinamento do Modelo

#### 7.1 Modelo de Regressão Logistica

In [9]:
# Treinar um modelo de Regressão Logística
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)
# Treinar um modelo de Regressão Logística
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

LogisticRegression()

#### 7.2 Resultados do modelo

In [10]:
# Fazer previsões nos dados de teste
y_pred = model.predict(X_test_tfidf)

# Avaliar a acurácia do modelo
print(f"Acurácia: {accuracy_score(y_test, y_pred)}")

# Exibir o relatório de classificação
print(classification_report(y_test, y_pred))

Acurácia: 0.9269591960019771
              precision    recall  f1-score   support

    negativo       0.85      0.64      0.73      2824
    positivo       0.94      0.98      0.96     15385

    accuracy                           0.93     18209
   macro avg       0.90      0.81      0.84     18209
weighted avg       0.92      0.93      0.92     18209



#### 7.3 Exportação do Modelo

In [16]:
import joblib

# Salvar o modelo treinado
joblib.dump(model, 'modelo_analise_sentimento.pkl')

['modelo_analise_sentimento.pkl']

#### 7.4 Treinando outro modelo

In [14]:
from sklearn.ensemble import RandomForestClassifier

# Treinar um modelo Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_resampled, y_train_resampled)

# Fazer previsões com o modelo Random Forest
y_pred_rf = rf_model.predict(X_test_tfidf)

# Avaliar o modelo
print(f"Acurácia (Random Forest): {accuracy_score(y_test, y_pred_rf)}")
print(classification_report(y_test, y_pred_rf))


Acurácia (Random Forest): 0.9191608545224889
              precision    recall  f1-score   support

    negativo       0.77      0.68      0.72      2824
    positivo       0.94      0.96      0.95     15385

    accuracy                           0.92     18209
   macro avg       0.86      0.82      0.84     18209
weighted avg       0.92      0.92      0.92     18209



A acurácia do modelo salvo (LogisticRegression) é de **92.7%**, no qual o resultado é considerado excelente. No entanto, analisando mais profundamente o relatório de classificação, podemos ver algumas nuances importantes:

### Detalhes do Relatório:
1. **Classe Negativa**:
   - **Precisão**: 0.85 → De todas as previsões que o modelo classificou como "negativo", 85% estavam corretas.
   - **Recall**: 0.64 → De todas as avaliações que realmente eram "negativas", o modelo conseguiu identificar 64%.
   - **F1-Score**: 0.73 → O equilíbrio entre precisão e recall, indicando que o modelo ainda tem margem de melhoria ao identificar corretamente os sentimentos negativos.

2. **Classe Positiva**:
   - **Precisão**: 0.94 → O modelo foi muito preciso ao identificar avaliações positivas.
   - **Recall**: 0.98 → Quase todas as avaliações positivas foram corretamente classificadas.
   - **F1-Score**: 0.96 → A pontuação F1 para as avaliações positivas é excelente.

3. **Média Macro e Média Ponderada**:
   - **Macro avg**: Dá uma média simples entre as classes, com um F1-Score de 0.84, sugerindo que o desempenho geral do modelo é equilibrado, mas a classe negativa está puxando um pouco o resultado para baixo.
   - **Weighted avg**: A média ponderada leva em consideração o desbalanceamento entre as classes, resultando em um F1-Score mais alto, que reflete o bom desempenho geral.

### O que significa:
- O modelo está muito bom para prever **comentários positivos**, mas encontra mais dificuldade em detectar **comentários negativos**.
- Isso provavelmente se deve ao desbalanceamento do conjunto de dados (15.385 avaliações positivas vs. 2.824 negativas), o que faz com que o modelo aprenda mais facilmente os padrões dos sentimentos positivos.